In [ ]:
# ==========================================
# SETUP BLOCK 
# ==========================================

import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


# ---- Imports from project files ----
sys.path.append(os.path.abspath(".."))
from Helper_funtions import (
    clean_up_subjects,
    calculate_true_false_score,
    calculate_Internet_terms_understanding_score,
    group_internet_understanding
)
from lists import (
    demographic_columns,
    multiple_choice_questions,
    single_choice_questions,
    likert_questions,
    likert_mapping,
    cross_tab_titles_and_colors

)
from answer_categories import question_orders

# ---- General plot style ----
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 5)

# ---- Data loading ----
DATA_FILE = os.path.join("..", "Data", "Fertige Tabelle.xlsx")
df = pd.read_excel(DATA_FILE)
df.columns = df.columns.str.strip()

# Clean up multi-subject columns
for col in ["Most used subjects", "Preferred Subjects", "Least preferred Subjects"]:
    if col in df.columns:
        df = clean_up_subjects(df, col)

# Calculate additional scores
if all(q in df.columns for q in ["True/False_1", "True/False_2"]):
    df = calculate_true_false_score(df)

if any(col.startswith("Internet terms_") for col in df.columns):
    df = calculate_Internet_terms_understanding_score(df)
    df = group_internet_understanding(df)

print("✅ Setup complete – DataFrame loaded and preprocessed")
print(f"Rows: {len(df)}, Columns: {len(df.columns)}")


In [ ]:
# --------- Creation of Clusters and Excel file "Clustered_Students" ----------

# Keep only rows that have *all* likert features present
df_clean = df.dropna(subset=likert_questions).copy()

# Replace Likert-scale labels with numeric values where needed.
# Numeric columns (like True_False_Score) will pass through unchanged.
df_numeric = (
    df_clean[likert_questions]
    .apply(lambda s: s.astype(object))          # kategoriellen dtype aufbrechen
    .replace(likert_mapping)
    .apply(pd.to_numeric, errors="coerce")      # sicher numerisch
)

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_numeric)

# Apply PCA
pca = PCA(n_components=2, random_state=42)
pca_result = pca.fit_transform(scaled_data)

# Show PCA loadings (influence of each question on the principal components)
components = pd.DataFrame(pca.components_, columns=likert_questions, index=["PC1", "PC2"])
print("📊 PCA loadings – influence of questions on components:")
print(components.T)

# Apply KMeans clustering
kmeans = KMeans(n_clusters=3, n_init=10, random_state=42)
clusters = kmeans.fit_predict(pca_result)

# Add clusters back to cleaned DataFrame
df_clean["Cluster"] = clusters

# Create a PCA scatter plot with clusters
df_plot = pd.DataFrame(pca_result, columns=["PC1", "PC2"])
df_plot["Cluster"] = clusters

swap_map = {0: 1, 1: 0}

# Neucodierung anwenden
df_clean["Cluster"] = df_clean["Cluster"].replace(swap_map)
df_plot["Cluster"]   = df_plot["Cluster"].replace(swap_map)

plt.figure(figsize=(10, 7))
sns.scatterplot(data=df_plot, x="PC1", y="PC2", hue="Cluster", palette="tab10", s=100)
plt.title("PCA of AI Attitudes (KMeans Clustering)", fontsize = 18)
plt.xlabel("Principal Component 1", fontsize = 14)
plt.ylabel("Principal Component 2", fontsize = 14)
plt.legend(title="Cluster", loc="best", fontsize = 12, title_fontsize=13)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


# Mean values per cluster 
cluster_means = (
    df_numeric.assign(Cluster=df_clean["Cluster"])
    .groupby("Cluster")
    .mean()
    .round(2)
)

print("\n📈 Mean scores of Likert questions per cluster:")
print(cluster_means.T)

# Plot heatmap of cluster means
plt.figure(figsize=(12, 8))
sns.heatmap(cluster_means.T, annot=True, cmap="YlGnBu", linewidths=0.5)
plt.title("Cluster Profiles based on Likert-scale Answers")
plt.xlabel("Cluster")
plt.ylabel("Question")
plt.tight_layout()
plt.show()


# --- Export: every CLuster as own Sheet in Clustered_Students.xlsx ---

# create file path
out_dir = os.path.join("..", "Data")
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, "Clustered_Students.xlsx")

# setting cluster as first column
export_cols = ["Cluster"] + [c for c in df_clean.columns if c != "Cluster"]
df_export = df_clean[export_cols].copy()

# writing
with pd.ExcelWriter(out_path, engine="openpyxl") as writer:  # engine kann auch "xlsxwriter" sein
    for c in sorted(df_export["Cluster"].unique()):
        sheet_name = f"Cluster_{c}"[:31]  # Excel-Limit: 31 Zeichen
        df_cluster = df_export[df_export["Cluster"] == c]
        df_cluster.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"✅ Exportiert: {out_path}")

In [ ]:
df_numeric[likert_questions].corr()

In [ ]:
# ------- Analysis of the seperate clusters -------

from IPython.display import display, HTML

# Dateien
cluster_file = os.path.join("..", "Data", "Clustered_Students.xlsx")
full_data_file = os.path.join("..", "Data", "Fertige Tabelle.xlsx")

# Daten laden
df_total = pd.read_excel(full_data_file)
df_total.columns = df_total.columns.str.strip()

# alle Cluster-Sheets laden (ein Sheet = ein Cluster)
xls = pd.read_excel(cluster_file, sheet_name=None)

def _to_str_clean(s):
    """Hilfsfunktion: NaNs droppen, zu String, trimmen."""
    return s.dropna().astype(str).str.strip()

def _explode_if_multiple(df, col):
    """Explode für Multiple-Choice-Spalten (kommagetrennt)."""
    tmp = df[[col]].copy()
    tmp = tmp.dropna()
    # falls Einträge schon Listen sind → direkt explodieren, sonst splitten
    if tmp[col].map(lambda x: isinstance(x, (list, tuple))).any():
        exploded = tmp.explode(col)
    else:
        exploded = tmp.assign(**{col: tmp[col].astype(str).str.split(",")}).explode(col)
    exploded[col] = exploded[col].astype(str).str.strip()
    exploded = exploded[exploded[col] != ""]
    return exploded[col]

def _apply_order(df, col):
    """Index nach question_orders (falls vorhanden) oder numerisch sortieren."""
    if col in question_orders:
        order = [str(v) for v in question_orders[col] if str(v) in df.index]
        return df.reindex(order)
    # numerische Sortierung, wenn Index rein numerisch ist
    if df.index.to_series().str.fullmatch(r"-?\d+(\.\d+)?").all():
        return df.sort_index(key=lambda x: x.astype(float))
    return df

# ---------- Analyse pro Cluster ----------
for sheet_name, cluster_df in xls.items():
    print(f"\n================  {sheet_name}  ================")

    for column in demographic_columns:
        if column not in df_total.columns:
            print(f"⚠️ Spalte '{column}' nicht im Gesamtdatensatz. Überspringe.")
            continue
        if column not in cluster_df.columns:
            print(f"⚠️ Spalte '{column}' nicht im Cluster-Sheet. Überspringe.")
            continue

        # --- Series vorbereiten (Single vs Multiple Choice) ---
        if column in multiple_choice_questions:
            cluster_series = _explode_if_multiple(cluster_df, column)
            total_series   = _explode_if_multiple(df_total, column)
        else:
            cluster_series = _to_str_clean(cluster_df[column])
            total_series   = _to_str_clean(df_total[column])

        # Kategorie-Level vereinheitlichen (alle Kategorien aus total als Basis)
        total_counts = total_series.value_counts(dropna=False)
        cluster_counts = cluster_series.value_counts(dropna=False)

        # --- (A) Cluster composition: % within the cluster ---
        # Share of each category within the given cluster
        cluster_comp_pct = (cluster_counts / cluster_counts.sum() * 100).reindex(total_counts.index).fillna(0).round(1)

        table_cluster_comp = pd.DataFrame({
            f"{sheet_name} Count": cluster_counts.reindex(total_counts.index).fillna(0).astype(int),
            f"{sheet_name} % (within cluster)": cluster_comp_pct
        })
        table_cluster_comp = _apply_order(table_cluster_comp, column)

        print(f"\n📊 {column} – Distribution in {sheet_name}")
        display(HTML(table_cluster_comp.style.format(precision=1).set_caption(f"{column} – {sheet_name}: Cluster-Komposition").to_html()))
        
        # Plot A
        plt.figure(figsize=(9, 4))
        sns.barplot(x=table_cluster_comp.index, y=table_cluster_comp[f"{sheet_name} % (within cluster)"], palette="Set2")
        plt.title(f"{column} – {sheet_name}: % of students in cluster")
        plt.xlabel(column)
        plt.ylabel("Prozent (%)")
        plt.xticks(rotation=45, ha="right")
        plt.ylim(0, 100)
        plt.tight_layout()
        plt.show()

        # --- (B) Category coverage of the cluster: % of the category in the cluster ---
        # Share of students with category k who are in this cluster
        # (Cluster_k / Total_k) * 100
        percent_of_category_in_cluster = (cluster_counts / total_counts * 100).reindex(total_counts.index).fillna(0).round(1)

        table_cat_capture = pd.DataFrame({
            "Total Count": total_counts.astype(int),
            f"{sheet_name} Count": cluster_counts.reindex(total_counts.index).fillna(0).astype(int),
            f"% of {column} in {sheet_name}": percent_of_category_in_cluster
        })
        table_cat_capture = _apply_order(table_cat_capture, column)

        print(f"\n📈 {column} – % of the category in {sheet_name}:")
        display(HTML(table_cat_capture.style.format(precision=1).set_caption(f"{column} – Anteil der Kategorie im {sheet_name}").to_html()))
        
        # Plot B
        plt.figure(figsize=(9, 4))
        sns.barplot(x=table_cat_capture.index, y=table_cat_capture[f"% of {column} in {sheet_name}"], palette="Set2")
        plt.title(f"{column} – % of the {column} category in {sheet_name}")
        plt.xlabel(column)
        plt.ylabel("Prozent (%)")
        plt.xticks(rotation=45, ha="right")
        plt.ylim(0, 100)
        plt.tight_layout()
        plt.show()


In [ ]:
# ------- Analysis of the separate clusters: Cluster vs Others (within clustering only) -------

import numpy as np
from IPython.display import display, HTML

# Dateien
cluster_file = os.path.join("..", "Data", "Clustered_Students.xlsx")
full_data_file = os.path.join("..", "Data", "Fertige Tabelle.xlsx")


df_total = df#pd.read_excel(full_data_file)
df_total.columns = df_total.columns.str.strip()

xls = pd.read_excel(cluster_file, sheet_name=None)

# --- Innerhalb-Clustering-Basis (nur Fälle, die im Clustering sind) ---
# ACHTUNG: setzt voraus, dass jedes Sheet dieselben Spalten wie df_total verwendet.
df_in_clustering = pd.concat(xls.values(), ignore_index=True)
df_in_clustering.columns = df_in_clustering.columns.str.strip()

def _to_str_clean(s: pd.Series) -> pd.Series:
    """NaNs droppen, zu String, trimmen."""
    return s.dropna().astype(str).str.strip()

def _explode_if_multiple(df: pd.DataFrame, col: str) -> pd.Series:
    """Explode für Multiple-Choice-Spalten (kommagetrennt)."""
    tmp = df[[col]].copy().dropna()
    if tmp[col].map(lambda x: isinstance(x, (list, tuple))).any():
        exploded = tmp.explode(col)
    else:
        exploded = tmp.assign(**{col: tmp[col].astype(str).str.split(",")}).explode(col)
    exploded[col] = exploded[col].astype(str).str.strip()
    exploded = exploded[exploded[col] != ""]
    return exploded[col]

def _apply_order(df_or_series, col: str):
    """Index nach question_orders (falls vorhanden) oder numerisch sortieren."""
    if 'question_orders' in globals() and isinstance(question_orders, dict) and col in question_orders:
        order = [str(v) for v in question_orders[col] if str(v) in (df_or_series.index if hasattr(df_or_series, 'index') else [])]
        if hasattr(df_or_series, 'reindex'):
            return df_or_series.reindex(order)
    # numerische Sortierung, wenn Index rein numerisch ist
    if hasattr(df_or_series, 'index'):
        idx = pd.Index(df_or_series.index)
        as_str = idx.astype(str)
        if pd.Series(as_str).str.fullmatch(r"-?\d+(\.\d+)?").all():
            return df_or_series.sort_index(key=lambda x: x.astype(float))
    return df_or_series

# Plot-Style (größere Fonts, konsistent)
TITLE_FSIZE = 18
LABEL_FSIZE = 14
TICK_FSIZE  = 12
LEG_FSIZE   = 12
LEG_TITLE_FSIZE = 13

BAR_PALETTE = "Set2"  # gerne anpassen (oder feste Farben setzen)

# ---------- Analyse pro Cluster-Sheet ----------
for sheet_name, cluster_df in xls.items():
    print(f"\n================  {sheet_name}  ================")

    
    if "Cluster" in cluster_df.columns and "Cluster" in df_in_clustering.columns:
        this_clusters = set(_to_str_clean(cluster_df["Cluster"]).unique())
        try:
            current_cluster_id = int("".join([c for c in sheet_name if c.isdigit()]))
            others_df = df_in_clustering[df_in_clustering["Cluster"] != current_cluster_id]
        except Exception:
            others_df = df_in_clustering.loc[~df_in_clustering.index.isin(cluster_df.index)]
    else:
        key_cols = list(set(df_in_clustering.columns).intersection(set(cluster_df.columns)))
        merged = cluster_df[key_cols].assign(_flag=1)
        others_df = df_in_clustering[key_cols].merge(merged, on=key_cols, how="left", indicator=False)
        others_df = df_in_clustering.loc[others_df["_flag"].isna()] if "_flag" in others_df.columns else df_in_clustering

    common_cols = set(df_total.columns) & set(cluster_df.columns) & set(df_in_clustering.columns)

    for column in demographic_columns:
        if column not in common_cols:
            print(f"⚠️ Spalte '{column}' fehlt in einer der Quellen (total/cluster/in_clustering). Überspringe.")
            continue

        if column in multiple_choice_questions:
            cluster_series = _explode_if_multiple(cluster_df, column)
            others_series  = _explode_if_multiple(others_df, column)
        else:
            cluster_series = _to_str_clean(cluster_df[column])
            others_series  = _to_str_clean(others_df[column])

        # Kategorie-Basis = alle Kategorien, die in Cluster oder Others vorkommen (innerhalb Clustering!)
        base_index = pd.Index(
            pd.unique(pd.concat([cluster_series, others_series], ignore_index=True))
        )

        # Zählungen
        cl_counts  = cluster_series.value_counts().reindex(base_index, fill_value=0)
        ot_counts  = others_series.value_counts().reindex(base_index, fill_value=0)

        # Prozent (spaltenweise, also innerhalb Gruppe)
        cl_pct = (cl_counts / cl_counts.sum() * 100).round(1) if cl_counts.sum() > 0 else cl_counts.astype(float)
        ot_pct = (ot_counts / ot_counts.sum() * 100).round(1) if ot_counts.sum() > 0 else ot_counts.astype(float)

        # Reihenfolge anwenden
        cl_counts = _apply_order(cl_counts, column)
        ot_counts = _apply_order(ot_counts, column)
        cl_pct    = _apply_order(cl_pct, column)
        ot_pct    = _apply_order(ot_pct, column)

        # --- Tabelle (A): Verteilung innerhalb Gruppe (%), Cluster vs Others ---
        table_comp = pd.DataFrame({
            f"{sheet_name} Count": cl_counts.astype(int),
            f"{sheet_name} % (within cluster)": cl_pct,
            "Others Count": ot_counts.astype(int),
            "Others % (within others)": ot_pct
        })

        print(f"\n📊 {column} – Cluster vs Others (within clustering)")
        display(HTML(
            table_comp.style
            .format(precision=1)
            .set_caption(f"{column} – Verteilung % innerhalb Gruppe: {sheet_name} vs Others")
            .to_html()
        ))

        # --- Plot (A): Prozentvergleich nebeneinander ---
        plot_df = pd.DataFrame({
            "Category": table_comp.index.astype(str),
            f"{sheet_name}": table_comp[f"{sheet_name} % (within cluster)"],
            "Others": table_comp["Others % (within others)"]
        }).melt(id_vars="Category", var_name="Group", value_name="Percent")

        plt.figure(figsize=(10, 5))
        sns.barplot(data=plot_df, x="Category", y="Percent", hue="Group", palette="Set2")
        plt.title(f"{column} – {sheet_name} vs Others (within-clustering %)", fontsize=TITLE_FSIZE)
        plt.xlabel(column, fontsize=LABEL_FSIZE)
        plt.ylabel("Percent (%)", fontsize=LABEL_FSIZE)
        plt.xticks(rotation=45, ha="right", fontsize=TICK_FSIZE)
        plt.yticks(fontsize=TICK_FSIZE)
        plt.legend(title="Group", fontsize=LEG_FSIZE, title_fontsize=LEG_TITLE_FSIZE, loc="best")
        plt.ylim(0, 100)
        plt.tight_layout()
        plt.show()

        # --- Tabelle (B): Kategorie-Abdeckung (Anteil einer Kategorie, der im Cluster liegt)
        # % of category in cluster = Cluster_k / (Cluster_k + Others_k) * 100
        denom = (cl_counts + ot_counts).replace(0, np.nan)
        pct_cat_in_cluster = (cl_counts / denom * 100).round(1).fillna(0)

        table_capture = pd.DataFrame({
            "Total in clustering (Count)": (cl_counts + ot_counts).astype(int),
            f"{sheet_name} (Count)": cl_counts.astype(int),
            f"% of {column} in {sheet_name}": pct_cat_in_cluster
        })

        print(f"\n📈 {column} – % of the category in {sheet_name} (within clustering)")
        display(HTML(
            table_capture.style
            .format(precision=1)
            .set_caption(f"{column} – Anteil der Kategorie in {sheet_name} (nur Clustering-Stichprobe)")
            .to_html()
        ))

        # --- Plot (B): Anteil einer Kategorie, der im Cluster liegt ---
        plt.figure(figsize=(10, 5))
        sns.barplot(x=table_capture.index.astype(str),
                    y=table_capture[f"% of {column} in {sheet_name}"],
                    palette=BAR_PALETTE)
        plt.title(f"{column} – % of the {column} category in {sheet_name}", fontsize=TITLE_FSIZE)
        plt.xlabel(column, fontsize=LABEL_FSIZE)
        plt.ylabel("Percent (%)", fontsize=LABEL_FSIZE)
        plt.xticks(rotation=45, ha="right", fontsize=TICK_FSIZE)
        plt.yticks(fontsize=TICK_FSIZE)
        plt.ylim(0, 100)
        plt.tight_layout()
        plt.show()


In [11]:
# -------- Age Distribution: Cluster 0 vs Other Clusters --------

plt.figure(figsize=(10, 5))

# Daten für Plot vorbereiten
plot_df = pd.DataFrame({
    "Age": table_comp.index.astype(str),
    "Cluster 0": table_comp["Cluster_0 % (within cluster)"],
    "Other Clusters": table_comp["Others % (within others)"]
}).melt(id_vars="Age", var_name="Group", value_name="Percent")

# Farben definieren: Blau & Orange
palette_map = {"Cluster 0": "#1f77b4", "Other Clusters": "#ff7f0e"}

ax = sns.barplot(
    data=plot_df,
    x="Age", y="Percent", hue="Group",
    palette=palette_map
)

# Titel & Achsen
ax.set_title("Age – Cluster 0 vs Other Clusters (within-clustering %)", fontsize=18)
ax.set_xlabel("Age", fontsize=15)
ax.set_ylabel("Percent (%)", fontsize=15)

# Ticks größer machen
ax.tick_params(axis="x", labelsize=13)
ax.tick_params(axis="y", labelsize=13)

# Y-Achse max. 50
ax.set_ylim(0, 50)

# Legende
ax.legend(title="Group", loc="upper right", fontsize=12, title_fontsize=13)

plt.tight_layout()
plt.show()


KeyError: 'Cluster_0 % (within cluster)'

<Figure size 1000x500 with 0 Axes>